In [ ]:
# Strip the header of raw files

from google.colab import files
import os
from datetime import datetime
from google.colab import drive
import shutil

def strip_header(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    modified_lines = lines[1:]  # Exclude the first line

    modified_content = ''.join(modified_lines)
    modified_file_path = os.path.splitext(file_path)[0] + '_stripped.txt'

    with open(modified_file_path, 'w') as modified_file:
        modified_file.write(modified_content)

    print(f'Stripped header and saved the modified file as: {modified_file_path}')

# Upload multiple files
uploaded_files = files.upload()

for file_name, file_content in uploaded_files.items():
    # Save the uploaded file
    with open(file_name, 'wb') as file:
        file.write(file_content)

    # Strip the header from the uploaded file
    strip_header(file_name)

    # Move the modified file to the specified Google Drive folder
    drive.mount('/content/drive')

    # Create a new folder with the current date in "My Drive/Colab Notebooks"
    base_folder = '/content/drive/My Drive/Colab Notebooks'
    current_date = datetime.now().strftime('%Y-%m-%d')
    folder_path = os.path.join(base_folder, current_date)
    os.makedirs(folder_path, exist_ok=True)

    # Move the modified file to the created folder
    modified_file_path = os.path.splitext(file_name)[0] + '_stripped.txt'
    new_file_path = os.path.join(folder_path, os.path.basename(modified_file_path))
    shutil.move(modified_file_path, new_file_path)

    print(f'Moved the modified file to: {new_file_path}')

In [ ]:
# Correct the iRu

from google.colab import files
import os
from datetime import datetime

# Get the current date
current_date = datetime.now().strftime('%Y-%m-%d')

# Create the output folder path
base_folder = '/content/drive/My Drive/Colab Notebooks'
folder_path = os.path.join(base_folder, current_date)
os.makedirs(folder_path, exist_ok=True)

# Upload the text file
uploaded = files.upload()

# Get the fill-in values from the user
reference_potential = float(input("Enter the reference potential (V): "))
resistance = float(input("Enter the resistance (Ω): "))
geometric_area = float(input("Enter the geometric area (cm²): "))

# Process the data
file_name = next(iter(uploaded))
file_path = os.path.join(folder_path, file_name)

processed_lines = []
with open(file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line:
            potential, current = line.strip().split('\t')
            new_current = float(current) / geometric_area
            new_potential = float(potential) - (float(current) / 1000 * resistance) + reference_potential
            processed_line = '{}\t{}\n'.format(new_potential, new_current)
            processed_lines.append(processed_line)

# Save the processed data to a new file with suffix '_iR-corrected.txt'
output_file_name = os.path.splitext(file_name)[0] + '_iR-corrected.txt'
output_file_path = os.path.join(folder_path, output_file_name)

with open(output_file_path, 'w') as output_file:
    output_file.writelines(processed_lines)

print('Data processing completed. Processed file saved as:', output_file_path)

In [ ]:
# Plot in .png, .svg, and .pdf

import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import matplotlib.ticker as ticker
import os
import shutil
from datetime import datetime
from google.colab import drive, files

def process_file(file_path):
    # Read the input file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extract x and y values from the file
    x = []
    y = []
    for line in lines:
        values = line.strip().split('\t')  # Split using tabs
        x.append(float(values[0]))
        y.append(float(values[1]))

    return x, y

def convert_to_subsuperscript(text, conversion_type):
    """
    Convert numerics in text to either subscript or superscript format.
    """
    subscript_dict = {
        '0': '₀', '1': '₁', '2': '₂', '3': '₃', '4': '₄',
        '5': '₅', '6': '₆', '7': '₇', '8': '₈', '9': '₉',
    }
    superscript_dict = {
        '0': '⁰', '1': '¹', '2': '²', '3': '³', '4': '⁴',
        '5': '⁵', '6': '⁶', '7': '⁷', '8': '⁸', '9': '⁹',
    }

    conversion_dict = {
        'sub': subscript_dict,
        'sup': superscript_dict,
    }

    converted_text = ''.join(conversion_dict.get(conversion_type, {}).get(char, char) for char in text)
    return converted_text

# Mount Google Drive
drive.mount('/content/drive')

# Select the files using a browse window
uploaded_files = files.upload()

fig, ax = plt.subplots()

# Set the aspect ratio of the plot
aspect_ratio = 3/5  # Width to height ratio
ax.set_box_aspect(aspect_ratio)

for file_name, file_content in uploaded_files.items():
    file_path = os.path.join(os.getcwd(), file_name)
    with open(file_path, 'wb') as file:
        file.write(file_content)

    x, y = process_file(file_path)

    # Get the file base name
    file_base = os.path.splitext(file_name)[0]

    # Extract the desired part of the file base name
    parts = file_base.split('_')
    legend_label_parts = [part for part in parts[1:] if part not in ['SCE', 'HgO', 'HgSO4', 'NaCl', 'MgSO4', 'Na2SO4', 'PBS', 'BBS', 'H2SO4', 'KOH', 'RT', '20C', '40C', '60C', '80C', '01', '02', '03', '04', '05', '06', '07', '08', 'PEIS', 'CV', 'CP', 'CA', 'stripped', 't-corrected', 'iR-corrected', 'merged']]
    legend_label = '-'.join(legend_label_parts)
    legend_label = [label.replace('-', '/') for label in legend_label]

    # Get the conversion type from the user
    conversion_type = input("Enter the conversion type (none/sub/sup): ")

    # Apply the conversion to the legend label
    legend_label = convert_to_subsuperscript(legend_label, conversion_type)

    # Plot the data
    ax.plot(x, y, label=legend_label)

# Set the axis labels
ax.set_xlabel('$\mathit{E - iR_u}$ / V vs. RHE')
ax.set_ylabel('$\mathit{j}_{\mathrm{geo}}$ / mA cm$^{-2}$')

# Configure ticks
ax.minorticks_on()
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(4))
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(4))
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))

# Set x-axis scale from lowest to highest value
#ax.set_xlim(min(x), max(x))

# Remove grid
ax.grid(False)

# Set the legend
ax.legend()

# Set the font size for axis labels and legend
plt.rcParams['font.size'] = 12

# Get the fill-in values from the user
electrolyte = input("Enter the electrolyte: ")
temperature = int(input("Enter the temperature (°C): "))
reference_electrode = input("Enter the reference electrode: ")

# Add annotations
annotations = [
    f'{electrolyte}',
    f'{temperature} °C',
    f'{reference_electrode}'
]

annotation_pos = (0.975, 0.025)  # Bottom right of the plot
for annotation in annotations:
    ax.annotate(annotation, xy=annotation_pos, xycoords='axes fraction', fontsize=12, ha='right', va='bottom')
    annotation_pos = (annotation_pos[0], annotation_pos[1] + 0.075)
    
# Save the plot as PNG, SVG, and PDF in the current date folder
current_date = datetime.now().strftime('%Y-%m-%d')
folder_path = os.path.join('/content/drive/My Drive/Colab Notebooks', current_date)
os.makedirs(folder_path, exist_ok=True)

plot_base_name = file_base + '_plot'
plot_file_path_png = os.path.join(folder_path, f'{plot_base_name}.png')
plot_file_path_svg = os.path.join(folder_path, f'{plot_base_name}.svg')
plot_file_path_pdf = os.path.join(folder_path, f'{plot_base_name}.pdf')

plt.savefig(plot_file_path_png, format='png', dpi=400)
plt.savefig(plot_file_path_svg, format='svg')
plt.savefig(plot_file_path_pdf, format='pdf')

print(f'Plot saved as: {plot_file_path_png}')
print(f'Plot saved as: {plot_file_path_svg}')
print(f'Plot saved as: {plot_file_path_pdf}')

# Move the plot files to the input file directory
input_file_dir = os.path.dirname(file_path)
shutil.copy(plot_file_path_png, os.path.join(input_file_dir, f'{plot_base_name}.png'))
shutil.copy(plot_file_path_svg, os.path.join(input_file_dir, f'{plot_base_name}.svg'))
shutil.copy(plot_file_path_pdf, os.path.join(input_file_dir, f'{plot_base_name}.pdf'))

print(f'Plot moved to: {os.path.join(input_file_dir, f"{plot_base_name}.png")}')
print(f'Plot moved to: {os.path.join(input_file_dir, f"{plot_base_name}.svg")}')
print(f'Plot moved to: {os.path.join(input_file_dir, f"{plot_base_name}.pdf")}')